<p style="text-align: center">
<img src="./images/landsat_8_rend-sm1.png" width=250 alt="Landsat 8"></img>
</p>

# Data Ingestion

---

## Overview

In this cookbook, you will access, process, analyze, and visualize satellite data in the context of example machine learning workflows. Machine learning tasks often involve a lot of data, and in Python, data is typically stored in memory as simple [NumPy](https://foundations.projectpythia.org/core/numpy.html) arrays. However, higher-level containers built on top of NumPy arrays provide more functionality for tabular data ([Pandas](http://pandas.pydata.org)), multidimensional gridded data ([xarray](http://xarray.pydata.org)), or out-of-core and distributed data ([Dask](http://dask.pydata.org)).

Likely, your approach to data access will depend not only on the intended analysis, but on features of the data, such as size and location. First, to demonstrate simple cases, we will begin by reading in a local files with Pandas and xarray. Then, as many earth science datasets are large, gridded, and stored on remote servers, we will demonstrate the combined use of [Intake](https://intake.readthedocs.io) and Dask libraries to efficiently fetch data.

Throughout this cookbook, we will use [HoloViz](https://holoviz.org/) visualization tools, such as [hvPlot](https://hvplot.holoviz.org/), [GeoViews](https://geoviews.org/), and [Datashader](https://datashader.org/), to efficiently explore and interact with the data at different steps in the pipeline.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Pandas](https://foundations.projectpythia.org/core/pandas.html) | Helpful |  |
| [xarray](https://foundations.projectpythia.org/core/xarray.html) | Necessary |  |
| [Intake](https://intake.readthedocs.io/en/latest/index.html) | Necessary |  |
|[Intake Cookbook](https://projectpythia.org/intake-cookbook/README.html)| Helpful | |

- **Time to learn**: 20 minutes

---

## Imports

In [ ]:
import pandas as pd
import xarray as xr
import intake
import hvplot.intake
import hvplot.xarray

import warnings
# Ignore a warning about the format of epsg codes
warnings.simplefilter('ignore', FutureWarning)

## About Landsat data

Before we dive into loading the data, let's review some of the key points about the dataset to build our intuition as we move toward processing and analysis.

The data in this cookbook come from the [Landsat](https://en.wikipedia.org/wiki/Landsat_program) program, which is the longest record of moderate resolution multispectral data of the Earth’s surface. This program has launched several different satellites spanning many years which are designated as Landsat 1-9. In this cookbook, we will use data from Landsat 5 and Landsat 8. Landsat 5 was launched in 1984 and retired in 2013, while Landsat 8 was launched in 2013 and will likely continue through at least mid 2023. 

<p style="text-align: center">
<img src = "images/landsat_timeline.png">
</p>
<p style="text-align: center">
Source: USGS - <a href="https://www.usgs.gov/landsat-missions/landsat-satellite-missions?qt-science_support_page_related_con=2#qt-science_support_page_related_con">USGS Landsat Timeline</a>
</p>


Landsat images are available from multiple sources and in different versions that have various levels of processing applied. However, a core aspect of this data is the use of different wavelength-bands to capture multiple images of the same area - together providing much more information than a single image alone. This provides us with a stack of images for each region that we might be interested, and therefore the first three dimensions for each chunk of data consist of the spatial coordinates (x and y) and the wavelength-band. Additionally, we also have a time dimension, as we will consider a stack of images from different years to look at the change in the water level around [Walker Lake](https://de.wikipedia.org/wiki/Walker_Lake) in Nevada, USA.



## Loading small and local data with Pandas

Now that we understand a bit about the context of the data, we'll start with the simple case of loading small local tabular data, such as a couple `.csv` files into Pandas Dataframes. 

First, let's load the Landsat 5 bands:

In [ ]:
landsat5_bands = pd.read_csv('data/landsat5_bands.csv').set_index('Band')
landsat5_bands

Now let's take a look at the relevant subset of Landsat 8 bands: 

In [ ]:
landsat8_bands = pd.read_csv('data/landsat8_bands.csv').set_index('Band')
landsat8_bands

Interestingly, we can see that the bands differ somewhat between the satellites. For example, 'Red' is Band-4 within Landsat 8, but Band-3 within Landsat 5. This will be important later on when we want to use the same bands from the different datasets.

## Loading small and local data with xarray

Although initiatives are underway to enhance cloud-based data workflows, at some point, it will likely be the case that you will have some local gridded data that you want to load and preview without much fuss. For this simple case, it makes sense to use an appropriately simple API. Using xarray and hvplot, we can quickly complete this task of previewing a [netCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) file.

**TODO: demonstrate hvplot.explorer when released**

In [ ]:
landsat_5_crop = xr.open_dataarray('data/landsat5_crop.nc')

In [ ]:
landsat_5_crop

In [ ]:
landsat_5_crop_image = landsat_5_crop.hvplot.image(geo=True, tiles='ESRI', alpha=.7)
landsat_5_crop_image

This image is just a cropped portion of a Landsat tile, solely for demonstration of xarray loading. Passing `geo=True` to [hvPlot](https://hvplot.holoviz.org/user_guide/Geographic_Data.html) will plot the data in a geographic coordinate system, allowing us to use the `tiles` option to overlay the plot on top of map tiles.

## Loading data of any size and location using Intake

Earth science data is often complex, so it's a good idea to become familiar with an approach that can handle data of nearly any size and location, even if your current data files are local. For this reason, we will demonstrate a couple different approaches of the [Intake](https://intake.readthedocs.io) library. 

First, we will start with a simpler case of reading the same local tabular file as above with Pandas. Then we will progressively cover more complex cases, such as loading multiple files and loading gridded data from a remote location with an Intake catalog.

In [ ]:
training = intake.open_csv('./data/landsat5_bands.csv')

Importantly, the data is not loaded yet. So far we only have the metadata:

In [ ]:
training

We can also peak at what format the data will be in when it gets loaded using `.container`:

In [ ]:
training.container

Now, to get better insight into the data without loading it all in just yet, we can inspect the data using `.to_dask()`. You could also use `.read_chunked()` to return an iterator over the chunks (or partitions) of data. Refer to the ['Reading Data'](https://intake.readthedocs.io/en/latest/quickstart.html?highlight=read_chunked#reading-data) Intake documentation for more information about these options.

In [ ]:
training_dd = training.to_dask()
training_dd.head()

In [ ]:
training_dd.info()

Notice that now running `.info()` gives us a lot less information than when we loaded the data earlier using `pd.read_csv`, this is because the `Dask` representation of the data only loads what it needs, when it is needed for a computation. However, you can always explicitly load all the data into memory using `.read()` (but be careful to only run this on a dataset that will fit on your computer's memory). Our sample dataset is very small, so let's try it:

In [ ]:
training_df = training.read()
training_df.info()

### Loading multiple local files

Intake also lets the user load and concatenate data across multiple files in one command

In [ ]:
training = intake.open_csv(['./data/landsat5_bands.csv', './data/landsat8_bands.csv'])

If the data file names allow, this can be more simply expressed using `*`:

In [ ]:
training = intake.open_csv('./data/landsat*_bands.csv')

In [ ]:
training_df = training.read()
training_df.info()

But sometimes there is data encoded in a file name or path that causes concatenated data to lose some important context. In this example, we lose the information about which version of landsat the training was done on (5 vs 8). To keep track of that information, we can use a [python format string](https://docs.python.org/3/library/string.html#format-string-syntax) `{variable:type}` to specify our path and declare a new field on our data. That field will get populated based on its value in the path.

In [ ]:
training = intake.open_csv('./data/landsat{Landsat:d}_bands.csv')
training_df = training.read().set_index(['Landsat', 'Band'])
training_df

### Loading remote data with a catalog

Probably the most versatile and reproducible approach to loading data with Intake is to use a simple Catalog file to declare how the data should be loaded. The catalog lays out how the data should be loaded, defines some metadata, and specifies any patterns in the file path that should be included in the data. Here is an example of an entry in a catalog YAML file:

``` yaml
sources:
  landsat_5_small:
    description: Small version of Landsat 5 Surface Reflectance Level-2 Science Product.
    driver: rasterio
    cache:
      - argkey: urlpath
        regex: 'earth-data/landsat'
        type: file
    args:
      urlpath: 's3://earth-data/landsat/small/LT05_L1TP_042033_19881022_20161001_01_T1_sr_band{band:d}.tif'
      chunks:
        band: 1
        x: 50
        y: 50
      concat_dim: band
      storage_options: {'anon': True}
```

The `urlpath` can be a path to a file, list of files, or a path with [glob](https://en.wikipedia.org/wiki/Glob_(programming)) notation. Alternatively, the path can be written as a python style [format_string](https://docs.python.org/3/library/string.html#format-string-syntax). In the case where the `urlpath` is a format string (for example, `{band:d}`), the fields specified in that string will be parsed from the filenames and returned in the data.

Let's look at what our catalog contains using `open_catalog`:

In [ ]:
cat = intake.open_catalog('./data/catalog.yml')
list(cat)

Here, we see a list of datasets specified by the catalog.yml file. Lets's look at a `description` and the `urlpath` for one of them:

In [ ]:
landsat_5 = cat.landsat_5_small

In [ ]:
landsat_5.description

In [ ]:
landsat_5.container

From `.container` we can see that ultimately the data will be an xarray object

In [ ]:
landsat_5.urlpath

From the `urlpath` we can see that this dataset is hosted remotely on Amazon S3

Let's preview this dataset using `.to_dask()`.

In [ ]:
landsat_5 = cat.landsat_5_small
landsat_5.to_dask()

From this information we can see that there are three dimensions with coordinates `x`, `y`, and `band`. This is satellite data where the `x` and `y` are spatial earth coordinates and the `band` refers to the wavelength range of light used to capture the satellite image.

### Visualizing the data

Often, data ingestion involves quickly visualizing your raw data to get a sense that things are proceeding accordingly. You have already seen how we can easily use [hvPlot](https://hvplot.pyviz.org/) for bare xarray data, but hvPlot also provides interactive plotting commands for Intake, Pandas, Dask, and GeoPandas objects. We'll look more closely at hvPlot and its options in later tutorials.

For now, let's just plot out our different band images as columns (using `col='band'`). We can also set `rasterize=True` to use [Datashader](https://datashader.org/) (another HoloViz tool) to render large data into a 2D histogram, where every array cell counts the data points falling into that pixel, as set by the resolution of your screen.

In [ ]:
landsat_5.hvplot.image(col='band', cmap='viridis', xaxis=False, yaxis=False, colorbar=False, rasterize=True)

Hint: Try zooming in on one plot to see that they are all linked!

If we increase the complexity a bit and consider the likely situation where you are constantly loading and visualizing a similar type of data using an Intake catalog, you can instead just declare plotting parameters directly into the catalog!

Here is the relevant part of our `catalog.yml`:

``` yaml
metadata:
  plots:
    band_image:
      kind: 'image'
      x: 'x'
      y: 'y'
      groupby: 'band'
      rasterize: True
      width: 400
      dynamic: False
```

In [ ]:
landsat_5 = cat.landsat_5_small
landsat_5.to_dask()

In [ ]:
landsat_5.hvplot.band_image()

Excellent! Since we didn't specify that the images should to be plotted as columns, hvPlot conveniently gives us a widget to scrub through them.

---

## Side-note: Drilling down for data access

As part of accessing data, it's important to understand how to drill down to different representations of our data that certain processing steps might require. As we saw earlier, Dask only loads the data into memory when it really needs it for a computation or when you explicitly ask, so let's go ahead and ask.

### Xarray DataArray

If the data are small enough, we can use the `.read()` method to read all the data into a regular `xarray.DataArray`. This method returns all the data into one in-memory container. Once in an `xarray` object the data can be more easily manipulated and visualized.

In [ ]:
landsat_5_xda = landsat_5.read()
type(landsat_5_xda)

In [ ]:
landsat_5_xda.head(3)

### Numpy Array

Under the hood, this xarray object is essentially comprised of NumPy arrays with the addition of coordinate labels (and [other superpowers](https://docs.xarray.dev/en/latest/getting-started-guide/why-xarray.html)). Sometimes, machine learning tools (e.g. [scikit-learn](https://scikit-learn.org/stable/)) accept NumPy arrays as input. These arrays are accessible to us from the xarray DataArray objects by using the `.values` attribute.

In [ ]:
landsat_5_npa = landsat_5_xda.values
type(landsat_5_npa)

In [ ]:
landsat_5_npa.shape

In [ ]:
landsat_5_npa[:3, :3, :3]

## Summary
Before accessing any data, it's a good idea to start by learning about the context and details of the dataset. This will give you the intuition to make informed decisions as you form a processing and analysis pipeline. Once you have identified a dataset to use, tailor the data access approach given the features of the data, such as size and location. If your data is local and small, a reliable approach is to access the data with either Pandas or xarray, depending on whether the data is tabular or gridded. As earth science data is often large, multidimensional, and on a remote server, a powerful and increasingly common approach is to use the combination of Intake, Dask, and xarray. Once you have accessed data, visualize it with hvPlot to ensure that it matches your expectations.

### What's next?
After loading the data, you will typically need to process it in some way before it can be fed into a machine learning pipeline. We will cover an example of such processing in the next section.

## Resources and references
- Authored/adapted by Demetris Roumis circa Dec, 2022
- This cookbook was inspired by the [EarthML](https://github.com/pyviz-topics/EarthML) tutorial. See a list of the EarthML contributors [here](https://github.com/pyviz-topics/EarthML/graphs/contributors).
- The landsat timeline image is originally from [USGS](https://www.usgs.gov/landsat-missions/landsat-satellite-missions?qt-science_support_page_related_con=2) but discovered through [earthsciencedata.org](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/)
- The landsat 8 banner image is from [NASA](https://svs.gsfc.nasa.gov/10812)
